In [1]:
from utils.dataloader import load_temusnews_data_raw
import textgrad as tg

data = load_temusnews_data_raw()

# Split data into train and test
split_ratio = 0.7
split_point = int(len(data) * split_ratio)
train_data = data[:split_point]
test_data = data[split_point:]

example = data[3]
example

{'month': 'june 2024',
 'id': 9,
 'full_text': '# dialogueforge: econcierge demos\nwe added a new module to dialogueforge - econcierge personas. meet maya, a front desk receptionist demoed for the national library board. maya is an energetic and witty personality who helps library users with events and wayfinding. testing this personality module has been so enjoyable that we often grab coffee before each session just to hang out with maya. ask for a demo today!',
 'short_text': "dialogueforge: econcierge\nmeet maya, a front desk receptionist that we created for a demo with the national library board. she's highly energetic, witty and knowledgeable and can help inform users about events and provide directions.\n\nmaya is a new persona that we have added to dialogueforge. connect with hardeep for a demo."}

In [2]:
system_prompt_string = tg.Variable(
    "Summarize the following article: \n",
    role_description="Instruction and style how to summarize the news article. Should be at most 50 words.",
    requires_grad=True,
)

tg.set_backward_engine("gpt-4o", override=True)
model = tg.BlackboxLLM("gpt-4o", system_prompt=system_prompt_string)
losses_list = []

optimizer = tg.TGD(parameters=[system_prompt_string])

In [6]:
def train_single(data, system_prompt_string, optimizer, model, losses_list, n_epochs=1):
    for k in range(n_epochs):
        for i, example in enumerate(data):
            news_string = tg.Variable(
                example["full_text"],
                role_description="String containing company monthly news that we want to summarize",
                requires_grad=False,
            )

            answer = model(news_string)
            answer.set_role_description(
                "A summary of the news article by the director of the company"
            )

            evaluation_instruction = (
                f"Here's a string containing news: {example['full_text']}. "
                f"Here is a string containing the summary of the director of the company: {example['short_text']}. "
                "You should evaluate the summaries to this news article in terms of syntax, style and personality. "
                "It should be as they were written by the same person."
            )

            loss_fn = tg.TextLoss(evaluation_instruction)
            loss = loss_fn(answer)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            losses_list.append(loss)

            # Print the info in a more visually separated and appealing way
            print(" ========= ")
            print(f"Epoch: {k + 1}, Iteration {i + 1}")
            print(" --------- ")
            print(f"Loss: {loss}")
            print(" --------- ")
            print(f"Prompt: {system_prompt_string}")
            print(" ========= ")


train_single(
    train_data, system_prompt_string, optimizer, model, losses_list, n_epochs=3
)

Epoch: 1, Iteration 1
 --------- 
Loss: The summaries provided for the news article and the director's statement are consistent in terms of syntax, style, and personality. Both summaries highlight the introduction of a new module on DataForge that allows users to interact with their data in Excel. They mention the positive reception from client demos and the interest in proceeding to a proof of concept (POC). The language used is clear and professional, focusing on the innovation and potential impact of the module. The director's statement adds a touch of enthusiasm by celebrating the module's innovation and recognition, which aligns with the positive tone of the news article. Overall, both summaries effectively convey the excitement and potential of the new DataForge module.
 --------- 
Prompt: Summarize the article in an enthusiastic, factual tone, using a mix of simple and complex sentences. Highlight innovation, awards, and community impact. Reflect the director's perspective, emph

KeyboardInterrupt: 

In [4]:
# Make an evaluation of the model for each example, showing the original text, the summary and the generated summary
def eval_model(model, data):
    for i, example in enumerate(data):
        news_string = example["full_text"]
        summary_string = example["short_text"]

        V_news_string = tg.Variable(
            news_string,
            role_description="String containing some news that we want to summarize",
            requires_grad=False,
        )

        answer = model(V_news_string)

        print("================================")
        print(f"Example {i + 1}")
        print(f"News: {news_string}")
        print("--------------------------------")
        print(f"Summary: {summary_string}")
        print("--------------------------------")
        print(f"Generated summary: {answer.value}")
        print("--------------------------------")
        print("\n")


eval_model(model, train_data)

Example 1
News: # DataForge: Talk to Your Data Demo
We rolled out a new module on DataForge that allows users to talk to their data in Excel. We demoed this to a client, who is keen to explore implementing this as a POC. Look out for more updates as we mature this accelerator!
--------------------------------
Summary: DataForge: Talk to your Data!
We have rolled out a new module on DataForge that allows us to talk with your data in Excel.

Recent client demos have created positive feedback with significant interest to proceed to a POC. Connect with Bram for a demo.
--------------------------------
Generated summary: DataForge recently introduced an innovative module enabling users to interact with their data in Excel through voice commands. This feature was demonstrated to a client interested in a proof of concept. The director emphasized the potential community impact, celebrating the module's innovation and its recognition through awards.
--------------------------------


Example 2


In [5]:
eval_model(model, test_data)

Example 1
News: # knowledgeforge grant evaluator: release day!
we are pleased to announce the completion of the build phase for the grant evaluator, and we are now entering the refinement phase. the temasek foundation team will be testing the evaluator in their next grant cycle with our support.
--------------------------------
Summary: temasek foundation grant evaluator: release day!
we recently completed the initial build of the grant evaluator powered by temus’ knowledgeforge. we are now entering the refinement phase where the foundation will be testing the evaluator on their next grant cycle working closely with our team.

this is a great example of how we’re deploying capabilities developed internally on talentforge to solve challenging client problems.
--------------------------------
Generated summary: The KnowledgeForge Grant Evaluator has successfully completed its build phase and is now in the refinement stage. Celebrating innovation, the Temasek Foundation team will test the